Build dataset


In [1]:
from typing import TypedDict, Type, Any, Callable

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from spark.config import views
from spark.create_session import create_session

from IPython.display import display

from fitter import Fitter, get_common_distributions

In [2]:
VIEWS = views("v3")
spark = create_session()

for view, file in VIEWS.items():
    df = spark.read.json(file)
    df.createOrReplaceTempView(view)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/26 20:27:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sessions_full = spark.sql(f"SELECT * FROM sessions").toPandas()
sessions = spark.sql(f"SELECT DISTINCT user_id, track_id FROM sessions WHERE event_type like 'like' order by user_id, track_id").toPandas()
tracks = spark.sql(
    f"SELECT DISTINCT id, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence FROM tracks ").toPandas()

In [4]:
display(sessions_full)
display(sessions)
display(tracks)

,event_type,session_id,timestamp,track_id,user_id
0,play,124,2023-09-20T07:18:17,4flBNxpPLGVAdufOzRBIen,101
1,like,124,2023-09-20T07:18:30.069000,4flBNxpPLGVAdufOzRBIen,101
2,play,125,2023-04-27T00:20:57.181000,3uMYq07Kj5m564OQwdSCrD,101
3,play,126,2023-02-11T05:34:54.160000,2RChe0r2cMoyOvuKobZy44,101
4,advertisment,126,2023-02-11T05:34:56.685000,None,101
...,...,...,...,...,...
580241,like,102015,2023-11-12T09:06:01.022000,4cLdpErILMO8Db8pQVAVcZ,1100
580242,play,102015,2023-11-12T09:09:17.564000,3j8ja2Hq824OaRqIENJPTH,1100
580243,like,102015,2023-11-12T09:12:19.739000,3j8ja2Hq824OaRqIENJPTH,1100
580244,skip,102015,2023-11-12T09:12:58.093000,3j8ja2Hq824OaRqIENJPTH,1100


,user_id,track_id
0,101,03LNdMgu3l3Ldc3QMl1bvZ
1,101,0BVCEJJFVsb8nrQGI11Dj2
2,101,0PCpQRd0hMAWjBLOmJdR7X
3,101,0bLOiofyBB62YU2cNnONJG
4,101,0iJfN2CqrX7O8hkzgAMMAf
...,...,...
80366,1100,7FdAQ7CXm9yS4DBtIKopLi
80367,1100,7a9aeLVkn7DIqFjbanKz0k
80368,1100,7an1exwMnfYRcdVQm0yDev
80369,1100,7cioKB5CHVzk09SOtTyn0T


,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,3KfbEIOC7YIv90FIfNSZpo,0.4490,0.688,146333,0.435,0.000000,9,0.1130,-11.359,74,1965,0.0323,103.239,0.435
1,1dxbAIfCASqv6jix2R1Taj,0.7840,0.698,148413,0.293,0.010600,0,0.0936,-11.361,55,1967,0.0332,117.613,0.724
2,17PXXzOygMyXXUNLngVN5u,0.8820,0.539,173733,0.114,0.009020,2,0.1060,-20.575,60,1970,0.0346,141.378,0.608
3,3Um9toULmYFGCpvaIPFw7l,0.4470,0.283,233000,0.716,0.000000,1,0.3990,-9.632,72,1971,0.0986,201.960,0.828
4,0GjEhVFGZW8afUYGChu3Rr,0.3580,0.543,230400,0.870,0.000939,9,0.7920,-6.514,81,1976,0.0428,100.804,0.754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,3VbsXTdxBl6IavGjFZbl3R,0.0394,0.720,225012,0.824,0.000000,10,0.0509,-5.086,53,2020,0.1280,165.998,0.333
22408,2zurBuX8TGOHkZLhye9Xme,0.0551,0.769,179080,0.922,0.000000,9,0.1940,-2.779,51,2020,0.1020,119.991,0.804
22409,3ev1ilsHdVbpXeT7FG6Kw7,0.6410,0.838,229187,0.461,0.000000,1,0.0825,-9.610,54,2017,0.2170,104.979,0.497
22410,5qTwnr6WbWeeUDS9dcl5g6,0.3370,0.444,303097,0.523,0.000000,1,0.1130,-4.522,54,2019,0.0286,185.872,0.420


In [5]:
# d = spark.sql(
#     """
#     SELECT s.user_id, s.track_id, s.weight, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence
#     FROM (
#         select user_id, track_id, sum(event_weight) as weight
#         from (
#             SELECT user_id, track_id, CASE WHEN event_type like 'like' THEN 1 ELSE 0.02 END as event_weight
#             FROM sessions
#             WHERE event_type like 'like' or event_type like 'play'
#             ) 
#         group by user_id, track_id
#     ) s
#     inner join tracks t on s.track_id = t.id
#     order by s.user_id, t.id
#     """).toPandas()
d = spark.sql(
    """
    SELECT s.user_id, s.track_id, s.weight, acousticness, danceability, duration_ms, energy, instrumentalness, key, liveness, loudness, popularity, EXTRACT(year from `release_date`) as release_year, speechiness, tempo, valence
    FROM (
        select user_id, track_id, sum(event_weight) as weight
        from (
            SELECT user_id, track_id, 1 as event_weight
            FROM sessions
            WHERE event_type like 'like'
            ) 
        group by user_id, track_id
    ) s
    inner join tracks t on s.track_id = t.id
    order by s.user_id, t.id
    """).toPandas()
d

,user_id,track_id,weight,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,101,03LNdMgu3l3Ldc3QMl1bvZ,1,0.54500,0.674,182549,0.6370,0.000002,0,0.0483,-3.400,52,2014,0.0330,139.867,0.755
1,101,0BVCEJJFVsb8nrQGI11Dj2,1,0.00522,0.656,235253,0.9230,0.000000,2,0.3260,-3.541,56,2011,0.0339,139.930,0.652
2,101,0PCpQRd0hMAWjBLOmJdR7X,1,0.02280,0.481,166040,0.8820,0.000220,2,0.1520,-5.068,51,1999,0.0407,141.721,0.718
3,101,0bLOiofyBB62YU2cNnONJG,1,0.32900,0.457,404933,0.4280,0.000002,1,0.1690,-9.796,55,1975,0.0283,129.980,0.151
4,101,0iJfN2CqrX7O8hkzgAMMAf,1,0.76700,0.222,355634,0.0555,0.923000,0,0.1180,-22.118,53,1994,0.0321,98.941,0.033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80366,1100,7FdAQ7CXm9yS4DBtIKopLi,1,0.88100,0.444,250880,0.2660,0.000000,9,0.1460,-9.001,56,2004,0.0366,130.430,0.404
80367,1100,7a9aeLVkn7DIqFjbanKz0k,1,0.31900,0.145,271867,0.4520,0.920000,2,0.0753,-13.560,55,2007,0.0448,82.297,0.146
80368,1100,7an1exwMnfYRcdVQm0yDev,1,0.21700,0.418,239013,0.4820,0.000000,5,0.1230,-5.769,54,2006,0.0266,175.558,0.261
80369,1100,7cioKB5CHVzk09SOtTyn0T,1,0.10300,0.436,369040,0.5340,0.006430,1,0.5070,-9.416,59,2014,0.0773,122.822,0.325


In [6]:
from scipy import stats
d_features = stats.zscore(d.drop(['user_id', 'track_id', 'weight'], axis=1))
d_features

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,popularity,release_year,speechiness,tempo,valence
0,0.998891,0.475696,-0.694795,-0.055256,-0.265465,-1.479128,-0.863278,1.010233,-1.162493,0.559228,-0.558926,0.607165,0.972939
1,-0.947493,0.361782,0.087430,1.302423,-0.265476,-0.918726,0.784774,0.972747,-0.666516,0.342740,-0.548871,0.609285,0.552061
2,-0.884102,-0.745717,-0.939819,1.107791,-0.264052,-0.918726,-0.247855,0.566784,-1.286487,-0.523216,-0.472903,0.669554,0.821750
3,0.220020,-0.897602,2.605795,-1.047407,-0.265462,-1.198927,-0.146966,-0.690186,-0.790510,-2.255126,-0.611433,0.274459,-1.495124
4,1.799398,-2.384814,1.874106,-2.815713,5.711417,-1.479128,-0.449633,-3.966072,-1.038499,-0.884030,-0.568980,-0.770029,-1.977295
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80366,2.210469,-0.979874,0.319364,-1.816442,-0.265476,1.042683,-0.283463,-0.478830,-0.666516,-0.162401,-0.518707,0.289602,-0.461316
80367,0.183961,-2.872114,0.630849,-0.933476,5.691990,-0.918726,-0.703042,-1.690871,-0.790510,0.054088,-0.427098,-1.330114,-1.515555
80368,-0.183839,-1.144416,0.143235,-0.791062,-0.265476,-0.078122,-0.419959,0.380419,-0.914504,-0.018075,-0.630425,1.808198,-1.045642
80369,-0.594910,-1.030502,2.073076,-0.544211,-0.223839,-1.198927,1.858945,-0.589161,-0.294533,0.559228,-0.064014,0.033587,-0.784126


In [7]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(
    users=d['user_id'],
    items=d['track_id'],
    item_features=d_features)

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 1000, num_items 21792.


In [8]:
(interactions, weights) = dataset.build_interactions(d[['user_id', 'track_id', 'weight']].apply(tuple, axis=1))

print(repr(interactions))

<1000x21792 sparse matrix of type '<class 'numpy.int32'>'
	with 80371 stored elements in COOrdinate format>


In [9]:
from lightfm.cross_validation import random_train_test_split

(train, test) = random_train_test_split(interactions)


In [10]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank


In [11]:
feature_names = d.drop(['user_id', 'track_id', 'weight'], axis=1).columns
# feature_names = ["danceability", "energy", "key", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

item_features = dataset.build_item_features(((i, feature_names) for i in d['track_id']))
print(repr(item_features))

<21792x21805 sparse matrix of type '<class 'numpy.float32'>'
	with 305088 stored elements in Compressed Sparse Row format>


In [12]:
model_h = LightFM(
    loss='warp-kos',
    k=3,
    learning_rate=0.02,
    item_alpha=1e-4,
    user_alpha=1e-4,
    no_components=30,
)
model_h.fit(
    interactions=train,
    item_features=item_features,
    epochs=50,
    num_threads=12)

In [13]:
train_auc_h = auc_score(model_h, train, item_features=item_features, num_threads=12).mean()
test_auc_h = auc_score(model_h, test, train_interactions=train, item_features=item_features, num_threads=12).mean()

train_precision_h = precision_at_k(model_h, train, k=10, item_features=item_features, num_threads=12).mean()
test_precision_h = precision_at_k(model_h, test, k=10, train_interactions=train, item_features=item_features, num_threads=12).mean()

train_recall_h = recall_at_k(model_h, train, k=10, item_features=item_features, num_threads=12).mean()
test_recall_h = recall_at_k(model_h, test, k=10, train_interactions=train, item_features=item_features, num_threads=12).mean()

train_reciprocal_rank_h = reciprocal_rank(model_h, train, item_features=item_features, num_threads=12).mean()
test_reciprocal_rank_h = reciprocal_rank(model_h, test, train_interactions=train, item_features=item_features, num_threads=12).mean()

print('AUC: train %.6f, test %.6f.' % (train_auc_h, test_auc_h))
print('Precision: train %.6f, test %.6f.' % (train_precision_h, test_precision_h))
print('Recall: train %.6f, test %.6f.' % (train_recall_h, test_recall_h))
print('Reciprocal rank: train %.6f, test %.6f.' % (train_reciprocal_rank_h, test_reciprocal_rank_h))


AUC: train 0.639561, test 0.615446.
Precision: train 0.009910, test 0.002424.
Recall: train 0.001637, test 0.001553.
Reciprocal rank: train 0.043242, test 0.011111.


In [14]:
from scipy.sparse import csr_matrix

i = np.zeros(interactions.shape)
i[148][:] = np.ones(interactions.shape[1]) - train.getrow(148)
i = csr_matrix(i)
w = csr_matrix(np.ones(item_features.shape))


In [22]:
predicted_ranks = model_h.predict_rank(i, item_features=item_features, train_interactions=train, num_threads=12)
predicted_ranks
predicted_scores = model_h.predict(148 * np.ones((interactions.shape[1])), np.arange(interactions.shape[1]), item_features=item_features, num_threads=12)
predicted_scores

array([31.796694, 31.83146 , 32.53239 , ..., 30.233364, 30.236204,
       30.247778], dtype=float32)

In [20]:
predicted_scores


KeyboardInterrupt: 